In [1]:
import numpy as np
import os
import cv2
import h5py
import tensorflow as tf

In [2]:
# If GPU is not available: 
# GPU_USE = '/cpu:0'
# config = tf.ConfigProto(device_count = {"GPU": 0})


# If GPU is available: 
config = tf.ConfigProto()
config.log_device_placement = True
config.allow_soft_placement = True
config.gpu_options.allocator_type = 'BFC'

# Limit the maximum memory used
config.gpu_options.per_process_gpu_memory_fraction = 0.01

# set session config
tf.keras.backend.set_session(tf.Session(config=config))

In [ ]:
# count_augment.npz file which is prepared from manual analysis, which consist of 
#count of encoding samples which have be augmented

data1 = np.load('/bigpool/Team3/count_augment.npz') 

#loading of labels, gaze and image frames and timestamps(not images)
data2 = np.load('/bigpool/Team3/P01_10_OutputLabels_timestamp.npz')
data3 = np.load('/bigpool/Team3/P01_10gaze30.npz')
data4 = np.load('/bigpool/Team3/P01_10_VideoFrames_Timestamp.npz')
daq = data1['a']
label = data2['label']
gaze30 = data3['inputs']
vid = data4['frames']

#code to select samples for data augmentation of required labels in a random way  

(rd,cd) = daq.shape
(rl,cl) = label.shape
label = label[label[:,0].argsort()]
u = np.unique(label[:,0])
da_full = []
for i in range(rd):
    print(i,"is going on")
    c = 0
    for j in range(rl):
        if np.array_equal(daq[i,(0,1,2,3,4,5,6,7,8)], label[j,(1,2,3,4,5,6,7,8,9)]):           
            da_full.append(label[j,:])
            c = c+1
    print(i," count:", c)
    da_select = random.sample(da_full, daq[i,9])
    da_full.clear()
    if i == 0:
        da = np.asarray(da_select)
    else:
        da_temp =  np.asarray(da_select)
        da = np.vstack((da,da_temp))           
                         

            
# taking the gaze data for augmentation of selected samples
(rda,cda) = da.shape
(l,m,n) = gaze30.shape
gaze30_da = np.zeros((rda,m,n))
e = 0
count = 0
da = da[da[:,0].argsort()]
for i in range(rda):
    print(i ,"is done out of ",rda )    
    for j in range(e,l):        
        if da[i,0] == gaze30[j,0,0]:
            count = count +1            
            gaze30_da[i,:,:] = gaze30[j,:,:] 
            e = j
            break     
    
print("total matched is : ",count )         


# taking the frame numbers(along with time stamps) for augmentation of selected samples
(rda,cda) = da.shape
(lv,mv) = vid.shape
vid_da = np.zeros((rda,mv))
vid = vid[vid[:,0].argsort()]
e = 0
count = 0
da = da[da[:,0].argsort()]
for i in range(rda):
    print(i ,"is done out of ",rda )    
    for j in range(e,lv):        
        if da[i,0] == vid[j,0]:
            count = count +1            
            vid_da[i,:] = vid[j,:] 
            e = j
            break     
print("total to be matched: ",rda)    
print("total matched is : ",count )

np.savez('/bigpool/Team3/data_augment_label.npz', label_da = da)
np.savez('/bigpool/Team3/data_augment_gaze30.npz', gaze30_da = gaze30_da)
np.savez('/bigpool/Team3/data_augment_vid.npz', vid_da = vid_da)

In [3]:
#the files data_augment_gaze30.npz contains the gaze data to be augmented
#data_augment_vid.npz contains timestamps of frames to be augmented
data1 = np.load("/bigpool/Team3/data_augment_gaze30.npz")
gaze_data = data1['gaze30_da']
data2 = np.load("/bigpool/Team3/data_augment_vid.npz")
video_frames = data2['vid_da']

print(gaze_data.shape,video_frames.shape)

# Gaze data augmentation

In [8]:
for i in range(gaze_data.shape[0]):
    gaze_data[i,:,1] = 1 - gaze_data[i,:,1]# Augment only gaze x and pupil x
    gaze_data[i,:,3] = 1 - gaze_data[i,:,3]

#create new hdf5 file to store Augmented Gaze data in a dataset
with h5py.File('/bigpool/Team3/Augmented_GazeData.hdf5','a') as f:
    gaze_dataset = f.create_dataset('Gaze', data = gaze_data)

np.savez('/bigpool/Team3/Gaze_DataAugmented.npz', gaze_data = gaze_data)#save augmented gaze data in .npz file

# Video frames augmentation

In [45]:
#folder name for each person's scene video
folder_list = ["P01_2014_05_20","P02_2014_05_24","P03_2014_05_26","P04_2014_05_29","P05_2014_05_30",
               "P06_2014_05_31","P07_2014_06_04","P08_2014_06_07","P09_2014_06_08","P10_2014_06_09"]
timestamp_limit = [1000000, 2000000,3000000,4000000,5000000,6000000,7000000,8000000,9000000,10000000]# modified timestamps range for each person

In [47]:
with h5py.File('/bigpool/Team3/Augmented_Frames.hdf5','a') as f:
    video_dataset = f.create_dataset('Video', (1,180,320,3), maxshape = (None,180,320,3))
    timestamp_dataset = f.create_dataset('Timestamp',(1,1), maxshape = (None,1))

In [48]:
#extract selected frame,resize and flip it vertically.
def frame(path):
    
    with h5py.File('/bigpool/Team3/Augmented_Frames.hdf5','a') as f:

        vid = cv2.VideoCapture(path)
        vid.set(1,video_frames[i,1])
        success,image = vid.read() 
        image = cv2.resize(image, (0,0), fx=0.25, fy=0.25)  #resize the image
        flip_image = cv2.flip(image, 1) #flip the image vertically
        flip_image = flip_image.reshape((1,180,320,3))
        print ("timestamp:",video_frames[i,0])
        f['Video'].resize((f['Video'].shape[0] + flip_image.shape[0]), axis = 0)
        f['Video'][-flip_image.shape[0]:] = flip_image
        f['Timestamp'].resize((f['Timestamp'].shape[0] + 1), axis = 0)
        f['Timestamp'][-1:] = video_frames[i,0]
    return

In [51]:

for t in range(len(timestamp_limit)):
    with h5py.File('/bigpool/Team3/Augmented_VideoP0%s.hdf5',(t+1),'a') as f:
        video_dataset = f.create_dataset('Video', (1,180,320,3), maxshape = (None,180,320,3))
        timestamp_dataset = f.create_dataset('Timestamp',(1,1), maxshape = (None,1))
    for i in range(video_frames.shape[0]):
        if timestamp_limit[t] <= video_frames[i,0] < timestamp_limit[t] + 1000000:
            path = os.path.join("/bigpool/UbiComp2015/Recordings/" + folder_list[t] +"/4Recording/world_Rec.mkv")
            frame(path)

timestamp1: 1025587.0
timestamp1: 1025588.0
timestamp1: 1025589.0
timestamp1: 1025590.0
timestamp1: 1025591.0
timestamp1: 1025592.0
timestamp1: 1025593.0
timestamp1: 1025594.0
timestamp1: 1025595.0
timestamp1: 1025596.0
timestamp1: 1025597.0
timestamp1: 1025598.0
timestamp1: 1025599.0
timestamp1: 1025600.0
timestamp1: 1025601.0
timestamp1: 1025602.0
timestamp1: 1025603.0
timestamp1: 1025604.0
timestamp1: 1025605.0
timestamp1: 1025606.0


KeyboardInterrupt: 

In [ ]:
#loading of the np file which have images of scene data 
data_v = np.load('/bigpool/Team3/P10_OneSec.npz')
video = data_v['small_images_np']


#file Delete_TimeStamps.npz consist of details of which samples to be deleted
data = np.load('/bigpool/Team3/Delete_TimeStamps.npz')
delet = data['delet']
data1 = np.load('/bigpool/Team3/P10_VideoFrames_Timestamp.npz')
frame = data1['frames']

delet_sort = np.sort(delet)

#deletion of certain image to balance the count
video_time = frame[:,0]
video_del = np.intersect1d(video_time, delet_sort, return_indices=True)
video_index = video_del[1]
frame_del_done = np.delete(frame,video_index , axis=0)
video_del_done = np.delete(video,video_index , axis=0)

#for checking the count
print(video.shape, "is the shape of video files before deletion")
print(video_del_done.shape, "is the shape of video files after deletion")
print(frame.shape, "is the shape of frames after deletion")
print(frame_del_done.shape, "is the shape of frames after deletion")

np.savez('/bigpool/Team3/P10_OneSec_afterDeletion.npz', video_del_done = video_del_done)
np.savez('/bigpool/Team3/P10_VideoFrames_Timestamp_after_del.npz', frame_del_done = frame_del_done)

#this same code can be used for deletion of gaze and labels with sight modifications like changing file directory names

# Addition of augmented data to original dataset

In [ ]:
#Addition of augmented gaze data to the original dataset

data1 = np.load('/bigpool/Team3/P01_10gaze30_afterDeletion.npz') #Original gaze data after deletion
gaze_delete = data1['gaze_del_done']

data2 = np.load('/bigpool/Team3/Gaze_DataAugmented.npz') #Augmented gaze data
gaze_augment = data2['gaze_data']

total_gaze = np.vstack((gaze_delete,gaze_augment))
for t in range(len(timestamp_limit)):
    gaze_person = []
    for i in range(total_gaze.shape[0]):
        if timestamp_limit[t] <= total_gaze[i,0,0] < timestamp_limit[t] + 1000000:
            gaze_person.append(total_gaze[i])
    gaze_person_np = np.asarray(gaze_person)
    print(gaze_person_np.shape)
    np.savez('/bigpool/Team3/OneSec/P0%s_OneSec_Gaze30_Final.npz',(t+1) gaze_person_np = gaze_person_np)

In [ ]:
#Addition of augmented labels to the original dataset

data3 = np.load('/bigpool/Team3/P01_10_OutputLabels_timestamp_afterDeletion.npz') #Original labels after deletion
label_delete = data3['label_del_done']

data4 = np.load('/bigpool/Team3/data_augment_label.npz') #Augmented frames
label_augment = data4['label_da']

total_label = np.vstack((label_delete,label_augment))
for t in range(len(timestamp_limit)):
label_person = []
    for i in range(total_label.shape[0]):
        if timestamp_limit[t] <= total_label[i,0,0] < timestamp_limit[t] + 1000000:
            label_person.append(total_label[i])
    label_person_np = np.asarray(label_person)
    np.savez('/bigpool/Team3/OneSec/P0%s_OneSec_Label_Final.npz',(t+1) label_person_np = label_person_np)


In [ ]:
#Addition of augmented frames to the original dataset
data1= np.load('/bigpool/Team3/P03_OneSec_afterDeletion.npz')# Original frames after deletion
input1 = data1['video_del_done']

with h5py.File('/bigpool/Team3/Augmented_VideoP03.hdf5','r') as f: #Augmented frames
    augment_data = f['Video_Frames'][:]
    
with h5py.File('/bigpool/Team3/Total_VideoP3.hdf5','a') as f:
    video_dataset = f.create_dataset('Total_Video', data = input1, maxshape = (None,180,320,3))
    f['Total_Video'].resize((f['Total_Video'].shape[0] + augment_data.shape[0]), axis = 0)
    f['Total_Video'][-augment_data.shape[0]:] = augment_data

